In [5]:
!pip install --upgrade pip

In [4]:
# Cell 1: ติดตั้งไลบรารี
import sys
!{sys.executable} -m pip install selenium webdriver_manager
print("ติดตั้งไลบรารีเสร็จสิ้น (หากติดตั้งไปแล้ว จะแสดงว่า Requirement already satisfied)")

  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl.metadata (1.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------- ----------------------- 3.9/9.4 MB 23.8 MB/s eta 0:00:01
   -------------------------------------- - 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import sys
print(sys.executable)

c:\Users\ball-pc\AppData\Local\Programs\Python\Python313\python.exe


In [1]:
# --- Cell 1: Imports และการตั้งค่า ---
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from dotenv import load_dotenv

# โหลดค่าจากไฟล์ .env
load_dotenv()

# --- ค่าคงที่ ---
LOGIN_URL = "https://cenproject.rid.go.th/"
YOUR_USERNAME = os.getenv("CEN_USERNAME")
YOUR_PASSWORD = os.getenv("CEN_PASSWORD")
EXCEL_FILE_PATH = 'data.xlsx' # <<<<---- ระบุชื่อไฟล์ Excel ของคุณที่นี่
OUTPUT_EXCEL_PATH = 'result_data.xlsx' # <<<<---- ไฟล์ผลลัพธ์ที่จะถูกสร้างขึ้น

print("การตั้งค่าเสร็จสิ้น")

การตั้งค่าเสร็จสิ้น


### เข้า Login

In [5]:
print(YOUR_USERNAME, YOUR_PASSWORD)

rid2ez2 wa792d


In [8]:
# --- Cell 2: เริ่มต้น WebDriver และ Login ---
print("กำลังเริ่มต้น WebDriver...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()

print(f"กำลังเข้าสู่หน้า Login: {LOGIN_URL}")
driver.get(LOGIN_URL)

wait = WebDriverWait(driver, 5) # เพิ่มเวลารอเป็น 20 วินาทีเผื่อเว็บโหลดช้า

try:
    # 1. Login
    print("กำลังรอหน้า Login...")
    username_field = wait.until(EC.presence_of_element_located((By.NAME, "Username")))
    password_field = wait.until(EC.presence_of_element_located((By.NAME, "Password")))
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))

    print("กำลังกรอกข้อมูล Login...")
    username_field.send_keys(YOUR_USERNAME)
    password_field.send_keys(YOUR_PASSWORD)
    login_button.click()
    print("Login สำเร็จ")

    # 2. กดเข้าระบบติดตาม
    print("กำลังรอลิงก์ 'ระบบติดตาม'...")
    system_tracking_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[./img[@alt='banner-track']]")))
    system_tracking_link.click()
    print("คลิก 'ระบบติดตาม' สำเร็จ")

    # --- ส่วนที่แก้ไข ---
    # 3. รอให้หน้ากาก Loading หายไปก่อน
    print("กำลังรอให้หน้า Dashboard โหลดเสร็จ (รอให้ Loading Overlay หายไป)...")
    loading_overlay_xpath = "//div[contains(@style, 'display: block') and contains(@style, 'z-index: 9999')]"
    # เราจะรอจนกว่า Element นี้จะ "มองไม่เห็น" (invisible)
    wait.until(EC.invisibility_of_element_located((By.XPATH, loading_overlay_xpath)))
    print("Loading Overlay หายไปแล้ว หน้าเว็บพร้อมใช้งาน")
except Exception as e:
    print(f"เกิดข้อผิดพลาดระหว่างการ Login หรือการนำทางเริ่มต้น: {e}")
    # driver.quit()

กำลังเริ่มต้น WebDriver...
กำลังเข้าสู่หน้า Login: https://cenproject.rid.go.th/
กำลังรอหน้า Login...
กำลังกรอกข้อมูล Login...
Login สำเร็จ
กำลังรอลิงก์ 'ระบบติดตาม'...
คลิก 'ระบบติดตาม' สำเร็จ
กำลังรอให้หน้า Dashboard โหลดเสร็จ (รอให้ Loading Overlay หายไป)...
Loading Overlay หายไปแล้ว หน้าเว็บพร้อมใช้งาน


### งบประมาณรายจ่ายประจำปี (พ.ร.บ.) 2568

In [16]:
# --- ใส่โค้ดนี้แทนที่ try...except บล็อกเดิมของคุณ ---

try:
    # 4. คลิกเมนูหลัก "งบประมาณ"
    print("กำลังรอเมนู 'งบประมาณ'...")
    budget_menu_left_nav = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'งบประมาณ')]")))
    budget_menu_left_nav.click()
    print("คลิกเมนู 'งบประมาณ' สำเร็จ")

    # 4.1. รอให้หน้า "งบประมาณ" หลักโหลดเสร็จ
    base_budget_url = "https://cenproject.rid.go.th/track/budget?BudgetYear=2025"
    wait.until(EC.url_to_be(base_budget_url))
    print(f"มาถึงหน้า 'งบประมาณ' หลักแล้ว: {driver.current_url}")

    # 4.2. สร้าง URL เป้าหมายและใช้ driver.get()
    target_url = base_budget_url + "&BudgetSourceID=1"
    print(f"กำลังนำทางไปยัง URL เป้าหมาย: {target_url}")
    driver.get(target_url)
    
    # 5. รอให้ตารางข้อมูลของหน้าเป้าหมายโหลดเสร็จ (ปรับปรุงจุดนี้)
    print("กำลังรอให้ตารางข้อมูลโครงการโหลด...")
    
    # ลองเพิ่มเวลาในการรอให้มากขึ้น เผื่อข้อมูลเยอะแล้วโหลดช้า
    long_wait = WebDriverWait(driver, 30) # สร้าง wait ใหม่ที่รอนานขึ้นเป็น 30 วินาที
    
    # เราจะรอ "Container" ของตารางแทน ID ที่อาจจะเปลี่ยนไป
    # ลองหา class ของ <div> ที่ครอบตารางทั้งหมด เช่น 'card-body' หรือ 'table-responsive'
    # **** ปรับแก้ 'card-body' เป็น class ที่ถูกต้องจากหน้าเว็บ ****
    table_container = long_wait.until(EC.presence_of_element_located((By.CLASS_NAME, "card-body")))
    
    # หลังจากเจอ Container แล้ว ค่อยเช็คว่ามีตารางอยู่ข้างในหรือไม่
    table_element = table_container.find_element(By.TAG_NAME, "table")
    
    print(f"หน้าตางบประมาณรายจ่ายประจำปี (พ.ร.บ.) โหลดเรียบร้อยแล้ว (พบตาราง ID: {table_element.get_attribute('id')})")

except Exception as e:
    # แก้ไขข้อความ Error ตามที่ต้องการ
    print("\n--- การนำทางสำเร็จ แต่เกิดข้อผิดพลาดในการรอ Element ---")
    print(f"ข้อผิดพลาด: ไม่สามารถหา Element ที่ต้องการได้ภายในเวลาที่กำหนด")
    print(f"URL ปัจจุบันที่เกิดปัญหา: {driver.current_url}")
    print("สาเหตุที่เป็นไปได้: ID หรือ Class ของตารางข้อมูลในหน้านี้อาจไม่ตรงกับที่ระบุในโค้ด")
    # ไม่ต้องแสดง Stacktrace เต็มๆ เพื่อให้ดูสะอาดตา
    # print(f"รายละเอียด: {e}") 

กำลังรอเมนู 'งบประมาณ'...
คลิกเมนู 'งบประมาณ' สำเร็จ
มาถึงหน้า 'งบประมาณ' หลักแล้ว: https://cenproject.rid.go.th/track/budget?BudgetYear=2025
กำลังนำทางไปยัง URL เป้าหมาย: https://cenproject.rid.go.th/track/budget?BudgetYear=2025&BudgetSourceID=1
กำลังรอให้ตารางข้อมูลโครงการโหลด...

--- การนำทางสำเร็จ แต่เกิดข้อผิดพลาดในการรอ Element ---
ข้อผิดพลาด: ไม่สามารถหา Element ที่ต้องการได้ภายในเวลาที่กำหนด
URL ปัจจุบันที่เกิดปัญหา: https://cenproject.rid.go.th/track/budget?BudgetYear=2025&BudgetSourceID=1
สาเหตุที่เป็นไปได้: ID หรือ Class ของตารางข้อมูลในหน้านี้อาจไม่ตรงกับที่ระบุในโค้ด
